In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input

In [2]:
data= pd.read_csv(r"C:\Users\hsynt\Downloads\Selcuk Abi\Selcuk Abi\VeriSeti2.csv")

In [3]:
# Özellik ve Hedef Ayrımı
X = data.drop(columns=['Target_10', 'adjclose'])
y = data['Target_10']

# Zaman Serisi Formatına Getirme
def create_sequences(data, target, seq_length):
    sequences, targets = [], []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
        targets.append(target[i + seq_length])
    return np.array(sequences), np.array(targets)

SEQ_LENGTH = 10
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_seq, y_seq = create_sequences(X_scaled, y.values, SEQ_LENGTH)

# Eğitim ve test kümelerinin oluşturulması
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42, stratify=y_seq)

# PCA ve LDA Veri Hazırlığı
X_flat = X_seq.reshape(X_seq.shape[0], -1)  # Düzleştirilmiş veri

pca = PCA(n_components=10)
X_train_pca = pca.fit_transform(X_train.reshape(X_train.shape[0], -1))
X_test_pca = pca.transform(X_test.reshape(X_test.shape[0], -1))

lda = LDA(n_components=1)
X_train_lda = lda.fit_transform(X_train.reshape(X_train.shape[0], -1), y_train)
X_test_lda = lda.transform(X_test.reshape(X_test.shape[0], -1))

# LSTM Modeli Tanımlama
def build_lstm_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(64, activation='relu', return_sequences=False),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Performans Hesaplama
results = []
threshold = 60  # Underfitting için eşik

for reduction, X_tr, X_te, y_tr, y_te in zip(
    ['No PCA/LDA', 'PCA', 'LDA'],
    [X_train, X_train_pca, X_train_lda],
    [X_test, X_test_pca, X_test_lda],
    [y_train, y_train, y_train],
    [y_test, y_test, y_test]
):
    if reduction == 'No PCA/LDA':
        X_tr = X_tr.reshape(X_tr.shape[0], X_tr.shape[1], X_tr.shape[2])  # 3D giriş
        X_te = X_te.reshape(X_te.shape[0], X_te.shape[1], X_te.shape[2])
    else:
        # PCA ve LDA için uygun giriş formatı
        X_tr = np.tile(X_tr, (1, SEQ_LENGTH)).reshape(X_tr.shape[0], SEQ_LENGTH, -1)
        X_te = np.tile(X_te, (1, SEQ_LENGTH)).reshape(X_te.shape[0], SEQ_LENGTH, -1)

    lstm_model = build_lstm_model(X_tr.shape[1:])
    history = lstm_model.fit(X_tr, y_tr, epochs=10, batch_size=32, verbose=1, validation_data=(X_te, y_te))

    # Eğitim ve Test Kayıpları
    train_acc = history.history['accuracy'][-1] * 100
    test_acc = history.history['val_accuracy'][-1] * 100
    train_loss = history.history['loss'][-1]
    test_loss = history.history['val_loss'][-1]

    # Aşırı ve Yetersiz Öğrenme Kontrolü
    overfit = train_acc - test_acc > 5  # %5'ten fazla fark varsa aşırı öğrenme
    underfit = train_acc < threshold and test_acc < threshold

    results.append({
        'Reduction': reduction,
        'Model': 'LSTM',
        'Train Accuracy (%)': train_acc,
        'Test Accuracy (%)': test_acc,
        'Train Loss': train_loss,
        'Test Loss': test_loss,
        'Overfitting': overfit,
        'Underfitting': underfit
    })

# Performans Sonuçlarını Görselleştirme
results_df = pd.DataFrame(results)

# Performans Sonuçları Tablosu
print("Performans Sonuçları Tablosu:")
print(results_df)

# Sonuçları Kaydetme
results_df.to_csv("lstm_overfitting_underfitting_results.csv", index=False)
print("\nSonuçlar 'lstm_overfitting_underfitting_results.csv' dosyasına kaydedildi.")

Epoch 1/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8221 - loss: 0.3952 - val_accuracy: 0.8626 - val_loss: 0.3163
Epoch 2/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8697 - loss: 0.3139 - val_accuracy: 0.8702 - val_loss: 0.3060
Epoch 3/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8730 - loss: 0.3063 - val_accuracy: 0.8748 - val_loss: 0.2959
Epoch 4/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8764 - loss: 0.2958 - val_accuracy: 0.8746 - val_loss: 0.2925
Epoch 5/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8830 - loss: 0.2842 - val_accuracy: 0.8805 - val_loss: 0.2892
Epoch 6/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8843 - loss: 0.2830 - val_accuracy: 0.8799 - val_loss: 0.2907
Epoch 7/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8846 - loss: 0.2781 - val_accuracy: 0.8753 - val_loss: 0.2952
Epoch 8/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8883 - loss: 0.2723 - 